# Featuretools Demo

Vijay, Kowshik, Rahul

Let us first install and import Featuretools

In [1]:
!pip install featuretools
import featuretools as ft

    100% |████████████████████████████████| 204kB 3.8MB/s ta 0:00:01
    100% |████████████████████████████████| 430kB 4.1MB/s ta 0:00:0111
    100% |████████████████████████████████| 5.4MB 4.5MB/s eta 0:00:011
    100% |████████████████████████████████| 716kB 5.6MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/fa/37/45185cb5abbc30d7257104c434fe0b07e5a195a6847506c074527aa599ec/Click-7.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 512kB 5.5MB/s ta 0:00:011
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /home/vijay/.cache/pip/wheels/86/36/f4/e1fc49f198c623b8dc2156e80011a52f19a959d538f6b1a5db
Successfully built psutil
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
requests 2.19.1 has requirement chardet<3.1.0,>=3.0.2, but you'll have chardet 2.3.0 which is incompatible.
  Found existing installation: psutil 5.4.7
    Uninstalling psutil-5.4.7:
      Successfully uninstalled psutil-5.4.7
  Found exis

Load sample dataset and check the contents

In [ ]:
data = ft.demo.load_mock_customer()

In [5]:
customers_df = data["customers"]
customers_df

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [6]:
sessions_df = data["sessions"]
sessions_df.sample

,session_id,customer_id,device,session_start
13,14,1,tablet,2014-01-01 03:28:00
6,7,3,tablet,2014-01-01 01:39:40
1,2,5,mobile,2014-01-01 00:17:20
28,29,1,mobile,2014-01-01 07:10:05
24,25,3,desktop,2014-01-01 05:59:40


In [7]:
transactions_df = data["transactions"]
transactions_df.sample(5)

,transaction_id,session_id,transaction_time,product_id,amount
74,232,5,2014-01-01 01:20:10,1,139.20
231,27,17,2014-01-01 04:10:15,2,90.79
434,36,31,2014-01-01 07:50:10,3,62.35
420,56,30,2014-01-01 07:35:00,3,72.70
54,444,4,2014-01-01 00:58:30,4,43.59


## We have raw data now. We need to perform two steps to transform the data to the format required by Featuretools.

1) Represent data as entities, by creating a dictionary of entity names which contains a tuple of the dataframe and its indices. 

2) Represent the relationship between entities using the format

(parent_entity, parent_variable, child_entity, child_variable)

In [8]:
entities = {
    "customers" : (customers_df, "customer_id"),
    "sessions" : (sessions_df, "session_id", "session_start"),
    "transactions" : (transactions_df, "transaction_id", "transaction_time")
}

relationships = [("sessions", "session_id", "transactions", "session_id"),
                 ("customers", "customer_id", "sessions", "customer_id")]

# Running Deep Feature Synthesis

DFS input: A set of entities, a list of relationships, and the “target_entity” to calculate features for
    
Output of DFS is a feature matrix and the corresponding list of feature defintions.

In [9]:
feature_matrix_customers, features_defs = ft.dfs(entities=entities,
                                                 relationships=relationships, 
                                                 target_entity="customers")
feature_matrix_customers

,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),...,NUM_UNIQUE(sessions.MODE(transactions.product_id)),NUM_UNIQUE(sessions.DAY(session_start)),NUM_UNIQUE(sessions.YEAR(session_start)),NUM_UNIQUE(sessions.MONTH(session_start)),NUM_UNIQUE(sessions.WEEKDAY(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.DAY(session_start)),MODE(sessions.YEAR(session_start)),MODE(sessions.MONTH(session_start)),MODE(sessions.WEEKDAY(session_start))
customer_id,,,,,,,,,,,,,,,,,,,,,
1,60091,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,...,4,1,1,1,1,4,1,2014,1,2
2,13244,7,3,desktop,7200.28,37.705178,146.81,0.098259,8.73,77.422366,...,4,1,1,1,1,3,1,2014,1,2
3,13244,6,3,desktop,6236.62,43.683296,149.15,0.418230,5.89,67.060430,...,4,1,1,1,1,1,1,2014,1,2
4,60091,8,3,mobile,8727.68,45.068765,149.95,-0.036348,5.73,80.070459,...,5,1,1,1,1,1,1,2014,1,2
5,60091,6,3,mobile,6349.66,44.095630,149.02,-0.025941,7.55,80.375443,...,5,1,1,1,1,3,1,2014,1,2


## All we did was specify our required target variable and pass the tables as input. Featuretools gives us a meaningful representation by creating new features from primitive data.